In [1]:
# importing modules
import numpy as np
import pandas as pd
import ast

In [2]:
filename = "Data\\LOBs\\UoB_Set01_2025-01-02LOBs.txt"
with open(filename, 'r') as f:
    file_data = f.readlines()

In [3]:
class LimitOrderBook:
    def __init__(self, text_data):
        self.data = text_data
        self.time_step = 0
        self.max_time_step = len(text_data)
        self.update()

    def update(self, update_time = True):
        temp_str = self.data[self.time_step]
        temp_str = temp_str.split(",")
        exh = temp_str[1]
        temp_str = ','.join([temp_str[0]]+temp_str[2:])
        temp_str = ast.literal_eval(temp_str)

        self.current_time = temp_str[0]
        self.bids = temp_str[1][0][1]
        self.asks = temp_str[1][1][1]

        if update_time:
            self.time_step += 1

    def backtest(self, agent, verbose = False):
        PnL = 0
        stock_held = 0
        self.time_step = 0
        self.update(False)

        while self.time_step < self.max_time_step:
            print(f"{self.time_step}/{self.max_time_step} | PnL = {PnL} | stock_held = {stock_held}", end = "\r")
            given_data = (self.bids, self.asks)
    
            command = agent(given_data)
    
            if command:
                if command[0] == "buy": # place buy order
                    volume = command[1]
                    price = command[2]
    
                    if len(self.asks) == 0:
                        if verbose:
                            print("0 ask orders cannot match buy")
                    else:
                        lowest_ask = self.asks[0][0]
                        if lowest_ask > price:
                            if verbose:
                                print(f"Bid {price} too low, could not be matched with ask {lowest_ask}")
                        else:
                            if verbose:
                                print(f"Bid {price} matched with ask {lowest_ask}")
                            stock_held += 1
                            PnL -= lowest_ask
                        
                elif command[0] == "sell": # place sell order
                    print("selling")
                    volume = command[1]
                    price = command[2]
    
                    if len(self.bids) == 0:
                        if verbose:
                            print("0 bid orders cannot match sell")
                    elif stock_held <= 0:
                        if verbose:
                            print("0 stock held, nothing to sell")
                    else:
                        highest_bid = self.bids[0][0]
                        if highest_bid < price:
                            if verbose:
                                print(f"Ask {price} too high, could not be matched with bid {highest_bid}")
                        else:
                            if verbose:
                                print(f"Ask {price} matched with bid {highest_bid}")
                            stock_held -= 1
                            PnL += highest_bid
                
                else:  # do nothing
                    print("Invalid Command, nothing placed")

            self.update()

        return PnL, stock_held

    def __str__(self):
        out = f"Time: {self.current_time}"
        out += "\n" + f"\tBids: {self.bids}"
        out += "\n" + f"\tAsks: {self.asks}"

        return out

def agent_random(data):
    rnd = np.random.randint(0,5)

    command = []
    if rnd == 0:
        command.append("buy")
    elif rnd == 1:
        command.append("sell")
    else:
        return None

    vol = 1
    price = np.random.randint(0,1000)
    command.append(vol)
    command.append(price)

    return command

In [4]:
LOB = LimitOrderBook(file_data)
agent = agent_random
agent(0)

['buy', 1, 201]

In [ ]:
LOB.backtest(agent)